# Chapter 11: str vs bytes - Text and Binary Data

Python 3 draws a sharp line between **text** (`str`) and **binary data** (`bytes`/`bytearray`). A `str` is a sequence of Unicode characters, while `bytes` is a sequence of integers in the range 0-255. Understanding the boundary between these types is essential for working with files, network protocols, and international text.

## Key Concepts
- **`str`**: Immutable sequence of Unicode characters (code points)
- **`bytes`**: Immutable sequence of integers 0-255
- **`bytearray`**: Mutable version of `bytes`
- **Encoding**: Converting `str` to `bytes` (`str.encode()`)
- **Decoding**: Converting `bytes` to `str` (`bytes.decode()`)

## str Objects: Unicode Character Sequences

A Python `str` holds Unicode text. Each element is a single character, and `len()` counts characters, not bytes. This means strings can represent text from any writing system.

In [ ]:
# str objects hold Unicode text
text: str = "Hello, 世界! 🌍"
print(f"Text: {text}")
print(f"Type: {type(text).__name__}")
print(f"Length (characters): {len(text)}")

# Each element is a single-character str
first_char = text[0]
print(f"\nFirst character: {first_char!r}")
print(f"Type of text[0]: {type(first_char).__name__}")
print(f"Is str: {isinstance(first_char, str)}")

# Unicode characters from different scripts
samples: dict[str, str] = {
    "Latin": "café",
    "Greek": "αβγδ",
    "CJK": "漢字",
    "Emoji": "🐍🎉",
}
for script, example in samples.items():
    print(f"{script}: {example!r} (len={len(example)})")

## bytes Objects: Integer Sequences (0-255)

A `bytes` object is an immutable sequence of integers, each in the range 0-255. Unlike `str`, indexing a `bytes` object returns an **integer**, not a single-byte `bytes` object.

In [ ]:
# bytes literals use the b"..." prefix
data: bytes = b"Hello"
print(f"Data: {data}")
print(f"Type: {type(data).__name__}")
print(f"Length: {len(data)}")

# KEY DIFFERENCE: indexing bytes returns an int!
print(f"\ndata[0] = {data[0]}  (int, ASCII value of 'H')")
print(f"Type of data[0]: {type(data[0]).__name__}")

# Compare with str indexing
text: str = "Hello"
print(f"\ntext[0] = {text[0]!r}  (str)")
print(f"Type of text[0]: {type(text[0]).__name__}")

# Slicing bytes returns bytes (not int)
print(f"\ndata[0:1] = {data[0:1]}  (bytes slice)")
print(f"Type of data[0:1]: {type(data[0:1]).__name__}")

In [ ]:
# bytes are sequences of integers 0-255
raw: bytes = bytes([72, 101, 108, 108, 111])
print(f"From integers: {raw}")
print(f"As list: {list(raw)}")

# Hex representation
hex_data: bytes = bytes.fromhex("48 65 6c 6c 6f")
print(f"\nFrom hex: {hex_data}")
print(f"To hex: {hex_data.hex(' ')}")

# bytes only allow ASCII characters in literals
valid: bytes = b"ASCII only: abc 123"
print(f"\nValid bytes literal: {valid}")

# Non-ASCII must use escape sequences
escaped: bytes = b"\xc3\xa9"  # UTF-8 encoding of 'é'
print(f"Escaped bytes: {escaped}")
print(f"Decoded: {escaped.decode('utf-8')}")

## Encoding: str.encode() -> bytes

Encoding converts text (`str`) into a binary representation (`bytes`) using a specified codec. UTF-8 is the dominant encoding on the modern web and the default in Python.

In [ ]:
# Encoding converts str -> bytes
text: str = "café"
encoded: bytes = text.encode("utf-8")

print(f"Original str: {text!r}")
print(f"Encoded bytes: {encoded}")
print(f"Byte values: {list(encoded)}")
print(f"str length: {len(text)} characters")
print(f"bytes length: {len(encoded)} bytes")
print(f"\nNote: 'é' takes 2 bytes in UTF-8 (\\xc3\\xa9)")

# Same string, different encodings
encodings: list[str] = ["utf-8", "latin-1", "utf-16", "utf-32"]
print(f"\nEncoding {text!r} with different codecs:")
for enc in encodings:
    result = text.encode(enc)
    print(f"  {enc:10s}: {result.hex(' '):30s} ({len(result)} bytes)")

## Decoding: bytes.decode() -> str

Decoding converts binary data (`bytes`) back into text (`str`). You must use the **same encoding** that was used to encode the data, or you may get garbled text or errors.

In [ ]:
# Decoding converts bytes -> str
raw: bytes = b"caf\xc3\xa9"
decoded: str = raw.decode("utf-8")

print(f"Raw bytes: {raw}")
print(f"Decoded str: {decoded!r}")

# Roundtrip: encode then decode gives back the original
original: str = "Ñoño"
roundtrip: str = original.encode("utf-8").decode("utf-8")
assert roundtrip == original
print(f"\nRoundtrip verified: {original!r} == {roundtrip!r}")

# Decoding with the WRONG encoding produces garbage (mojibake)
utf8_bytes: bytes = "café".encode("utf-8")
wrong_decode: str = utf8_bytes.decode("latin-1")  # Wrong!
print(f"\nCorrect (utf-8):  {utf8_bytes.decode('utf-8')!r}")
print(f"Wrong (latin-1):  {wrong_decode!r}  <- mojibake!")

## UTF-8 Roundtrip Encoding and Decoding

UTF-8 is the most common encoding. It uses a variable number of bytes per character (1-4 bytes) and is backward-compatible with ASCII.

In [ ]:
# UTF-8 variable-length encoding demonstration
chars: list[tuple[str, str]] = [
    ("A", "ASCII letter"),
    ("é", "Latin accent"),
    ("中", "CJK character"),
    ("🎉", "Emoji"),
]

print(f"{'Char':<6} {'Description':<16} {'Code Point':<12} {'UTF-8 Bytes':<16} {'Byte Count'}")
print("-" * 65)
for char, desc in chars:
    encoded = char.encode("utf-8")
    codepoint = f"U+{ord(char):04X}"
    byte_hex = encoded.hex(" ")
    print(f"{char:<6} {desc:<16} {codepoint:<12} {byte_hex:<16} {len(encoded)}")

# Roundtrip all characters
mixed: str = "Aé中🎉"
assert mixed.encode("utf-8").decode("utf-8") == mixed
print(f"\nRoundtrip of {mixed!r}: OK")
print(f"String length: {len(mixed)} characters")
print(f"UTF-8 length: {len(mixed.encode('utf-8'))} bytes")

## bytearray: Mutable Bytes

`bytearray` is the mutable counterpart to `bytes`. You can modify individual byte values in place, which is useful for building binary data incrementally or for in-place transformations.

In [ ]:
# bytearray is mutable bytes
data = bytearray(b"hello")
print(f"Original: {data}")
print(f"Type: {type(data).__name__}")

# Modify in place
data[0] = ord("H")  # Replace 'h' with 'H'
print(f"After data[0] = ord('H'): {data}")

# Append and extend
data.append(ord("!"))
print(f"After append: {data}")

data.extend(b" World")
print(f"After extend: {data}")

# Convert between bytes and bytearray
frozen: bytes = bytes(data)  # bytearray -> bytes (immutable copy)
thawed: bytearray = bytearray(frozen)  # bytes -> bytearray (mutable copy)
print(f"\nFrozen (bytes): {frozen}")
print(f"Thawed (bytearray): {thawed}")
print(f"Equal: {frozen == thawed}")

## Practical Example: Text Files vs Binary Files

When opening files, the mode determines whether you get `str` or `bytes`. Text mode (`'r'`/`'w'`) handles encoding automatically. Binary mode (`'rb'`/`'wb'`) gives you raw bytes.

In [ ]:
import tempfile
import os

# Write text to a file, then read it both ways
content: str = "Hello, 世界! café\n"

with tempfile.NamedTemporaryFile(mode="w", suffix=".txt", delete=False, encoding="utf-8") as f:
    f.write(content)
    temp_path: str = f.name

try:
    # Text mode: returns str, handles encoding
    with open(temp_path, "r", encoding="utf-8") as f:
        text_data = f.read()
    print(f"Text mode type: {type(text_data).__name__}")
    print(f"Text mode data: {text_data!r}")

    # Binary mode: returns bytes, raw data
    with open(temp_path, "rb") as f:
        binary_data = f.read()
    print(f"\nBinary mode type: {type(binary_data).__name__}")
    print(f"Binary mode data: {binary_data!r}")
    print(f"Binary mode hex:  {binary_data.hex(' ')}")

    # Verify they represent the same content
    assert binary_data.decode("utf-8") == text_data
    print("\nText and binary represent the same content.")
finally:
    os.unlink(temp_path)

## Key Differences: str vs bytes Indexing

One of the most important distinctions to remember: indexing a `str` returns a `str`, but indexing `bytes` returns an `int`. This catches many people off guard.

In [ ]:
# Side-by-side comparison of str vs bytes behavior
text: str = "Python"
data: bytes = b"Python"

print(f"{'Operation':<25} {'str result':<20} {'bytes result':<20}")
print("-" * 65)
print(f"{'type(x)':<25} {type(text).__name__:<20} {type(data).__name__:<20}")
print(f"{'len(x)':<25} {len(text):<20} {len(data):<20}")
print(f"{'x[0]':<25} {text[0]!r:<20} {data[0]!r:<20}")
print(f"{'type(x[0])':<25} {type(text[0]).__name__:<20} {type(data[0]).__name__:<20}")
print(f"{'x[0:1]':<25} {text[0:1]!r:<20} {data[0:1]!r:<20}")
print(f"{'type(x[0:1])':<25} {type(text[0:1]).__name__:<20} {type(data[0:1]).__name__:<20}")

# Iteration comparison
print("\nIterating over str:")
for ch in text[:3]:
    print(f"  {ch!r} (type: {type(ch).__name__})")

print("\nIterating over bytes:")
for byte in data[:3]:
    print(f"  {byte} (type: {type(byte).__name__})")

In [ ]:
# Common operations available on both str and bytes
text: str = "Hello World"
data: bytes = b"Hello World"

# Many string methods work on both types
print("Shared methods:")
print(f"  text.split()  = {text.split()}")
print(f"  data.split()  = {data.split()}")
print(f"  text.upper()  = {text.upper()}")
print(f"  data.upper()  = {data.upper()}")
print(f"  text.replace('o', '0')  = {text.replace('o', '0')}")
print(f"  data.replace(b'o', b'0') = {data.replace(b'o', b'0')}")

# But you cannot mix str and bytes
try:
    text + data  # type: ignore[operator]
except TypeError as e:
    print(f"\nCannot mix str and bytes: {e}")

try:
    text.replace(b'o', b'0')  # type: ignore[arg-type]
except TypeError as e:
    print(f"Cannot use bytes arg on str: {e}")

## Summary

### Key Takeaways

- **`str`** holds Unicode text; `len()` counts characters, indexing returns `str`
- **`bytes`** holds raw binary data; `len()` counts bytes, indexing returns `int`
- **`bytearray`** is a mutable version of `bytes` for in-place modifications
- **Encoding** (`str.encode()`) converts text to bytes; **decoding** (`bytes.decode()`) converts bytes to text
- Always specify the encoding explicitly; UTF-8 is the most common choice
- Decoding with the wrong codec produces **mojibake** (garbled text)
- **Text mode** (`open('r')`) returns `str` and handles encoding; **binary mode** (`open('rb')`) returns `bytes`
- You **cannot mix** `str` and `bytes` in operations -- Python 3 enforces the boundary strictly